## Sharpening ECOSTRESS LST using Harmonized Landsat Sentinel (HLS) VSWIR products (30m)

Notebook : Quentin Dehaene, mentored by Glynn Hulley    
Original Python Implementation : [Radoslaw Guzinski](https://github.com/radosuav/pyDMS)  
Original Implemenation : [Gao et al.](https://doi.org/10.3390/rs4113287) 

Questions : quentin.dehaene@jpl.nasa.gov   

In [ ]:
# Import cell
from osgeo import gdal
import rasterio
import numpy as np
import os
import matplotlib.pyplot as plt
from rasterio.warp import calculate_default_transform, reproject, Resampling
import rasterio.mask
import rasterio.windows
import random
from pyDMS_master.pyDMS_master.run_pyDMS import *
# you will also need ipykernel to run this a python notebook

In this notebook, HR will refer to the high-resolution image used to train our regression algorithm. LR will refer to the coarse resolution image that we are trying to upsample, in our case the ECOSTRESS LST at 70m.

The first step is to reproject and subset LR to HR Coordinates Reference System (CRS) and extent. This means that the result, and all images in the process will be in the HR CRS and will share its extent. Even if it implies padding a smaller LR with nodata values to match the extents. Reversely, all the images bigger than HR would be cut.

The second step consists in resampling HR to the coarse resolution of LR (producing a temporary file). In the process, we compute the homogeneity inside each coarse resolution pixel. We are computing here how “pure” each coarse pixel is, i.e. how different are high resolution pixels that compose the coarse resolution pixel. For instance, if a coarse pixel is on a field, then it is very likely that all the high-resolution pixels composing that coarse pixel will be very similar because the material will be homogenous. However, in cities, larger pixels may actually be composed of several materials (asphalt roof, sidewalk, vegetation) and therefore be far less homogenous. We’ll use this homogeneity as a weight factor in our training.

The default threshold is 80%, meaning that the 20% of pixels that are the least homogenous will be disregarded for the training.

We can now start the training and fit the resampled HR to the LR.

For those interested in how the tree is conceived:

We are training a bagging of an ensemble of regression trees. Each of these trees is slightly more complicated than the usual regression tree. The principle of the tree is classic: each node is built to minimize the mean square error (MSE), hence each rule is made to split the data into different categories (the number of leaves being a parameter chosen here) and then we affect a value to each datum (each pixel in our case) falling into that category. The difference here lays in the way we determine the target value on each leaf: instead of giving the average of the y values (i.e. the LST) to all features in the leaf, we apply a Bayesian linear regression. We also have a limit as to how far from the training samples we can extrapolate when we predict with our regression tree.

The tree is thus trained using the resampled HR as $X_{train}$ and the reprojected LR as $y_{train}$ (ground truth). Our tree is trained at low resolution, because it is at this resolution that we have what we know to be true and that we can actually establish a link between reflectance and LST.

Now that the tree is trained, we can use it to make predictions. For that, we use the provided HR as X. We obtain a first prediction of the LST ($y_{pred}$) at the high resolution.

Then, comes the residual analysis: We compute the residual (y-$y_{pred}$) at the coarse resolution. We compare our predicted LST, resampled at the coarse resolution, to the original LR. For that, we don’t actually compare the LST but the $LST^4$ which is proportional to the exitance. It makes more sense, since physically, temperature doesn’t have to be conserved in the sharpening, the energy does. And sensors don’t measure temperature but radiance.

The final step is to smooth the residual, resample it to the high resolution, and sum it with the $y_{pred}$, our LST predicted. We now have a final LST prediction that verifies that the average radiance of all the high-resolution pixels composing a coarse pixel is equal to the radiance in the original LST.

This is our downscaled image that we were looking for.

I would like to note that there are two other sharpening techniques implemented in the pyDMS code, developed (and still being developed as of July 2024) by R. Guzinski. All the steps are the same, the only difference is the regression tree itself. There is on one hand, the Neural Network regressor i.e. instead of the tree presented here, we are using an MLP tree from [scikit-learn](https://github.com/aigamedev/scikit-neuralnetwork). On the other hand, is the one that’s actually closer to the original proposition of Gao, it is based on a [Cubist](https://www.rulequest.com/cubist-unix.html) regressor.

The results don’t differ significantly from what I have tried, I went to the oldest/default (and hence more fool-proofed) but I welcome any feedback from your end.


Path defining cell. When you type a directory, it shouldn't include the final / or \ (otherwise you will get a syntax error).

In [ ]:
# Folder where all the HLS bands are located.This can remain empty if you already have a multiband HLS.
hr_img_dir = r'' 
# Future path (and name) of the multiband HLS image to be used for sharpening. Pay attention to potential overwriting of previous HLS image.
hr_img_path = r'' 
# Folder where all the ECOSTRESS Quality Control files are located. This can remain empty if you don't have any QC file.
QC_dir = r''
# Folder where all the ECOSTRESS LST files are located for the scene of interest. This can remain empty if your ECOSTRESS data is already scaled.
lr_dir = r''
# Folder where all the scaled ECOSTRESS LST files will be written for the scene of interest
lr_dir_sc = r''  
# Folder where all the sharpened ECOSTRESS LST files will be written for the scene of interest
dst_dir = r''

### HLS preprocessing

This part assumes that you already have on you computer/server the HLS data product for the region of interest.

Skip this cell if you have a single HLS raster that is already scaled and with muliple bands (should be 10 if it's a Sentinel product, 8 if it is Landsat).

In [ ]:
# Make sure that the band per band reflectances already exist in EPSG:4326' 

r = []
for root, dirs, files in os.walk(hr_img_dir):
    for name in files:
        if name.endswith('RP_cropped.tif') : # the naming convention might differ, you need this to read the mozaic HLS tile over the expected area
            path =  os.path.join(root, name) 
            if path.__contains__('S30') and not  (path.__contains__('Band_01') or path.__contains__('Band_09') or path.__contains__('Band_10') or path.__contains__('Band_05') or path.__contains__('Band_06')  or path.__contains__('Band_07')  or path.__contains__('Band_8A')or path.__contains__('Band_11')or path.__contains__('Band_12')) :                        
                r.append(path)
            elif path.__contains__('L30') and not  (path.__contains__('Band_10') or path.__contains__('Band_11')):
                r.append(path)
            
nb_bands = len(r)            
print('Number of bands: ',nb_bands) # Check that it's what's expected 
         
reproj_list = list()
refl_hls = list() 

for filename in r : 
    with rasterio.open(filename,'r') as solo_band_rp :
        refl_hls.append(solo_band_rp.read().astype('int16')[0])
        kwargs = solo_band_rp.meta.copy()
        kwargs.update({'count': nb_bands,'dtype' :'float32'})
    # The scaling is different if the HLS comes from Sentinel or Landsat
    if filename.__contains__('S30') : 
        scale_list = [0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001]
    elif filename.__contains__('L30') : 
        scale_list = [0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001]
    else :
        print('Scaling was not correctly performed, check the names of the single bands mozaics' )

        
with rasterio.open(hr_img_path, 'w', **kwargs) as dst:
    for i in range (nb_bands) :
        dst.write(refl_hls[i]*scale_list[i], i+1)
        

### ECOSTRESS preprocessing

Here too we assume that you have the ECOSTRESS data dowloaded alongside the Quality Control files if available. In the folder there's a tutorial on how to use [AppEEARS](https://appeears.earthdatacloud.nasa.gov/) and obtain it.  
Preprocessing the QC files to make it usable by pyDMS.  
Skip this cell if you have no QC file or if they already been processed previously.

In [ ]:

for file in os.listdir(QC_dir) :
    if not file.endswith('QF.tif') and not file.endswith('.xml') : # avoid the auxiliary files created by QGIS and avoid creating QF files using already created QF files
        file_qc = os.path.join(QC_dir,file)
        with rasterio.open(file_qc,'r') as f_qc :
            qc_img = f_qc.read((1)) # read the QC file they are coded in 16 bits
            qc_img[qc_img==-99999] = -1  # Nodata values are read as -99999, we change it to -1 so that the last two bits appear as 11 (which means pixel not produced) and be masked out in the end
            qc_img_2 = qc_img & 0b11 # select only the last two bits
            out_meta = f_qc.meta.copy()
        
        file_qf = file_qc.replace('.tif','_QF.tif')
        with rasterio.open(file_qf,'w',**out_meta) as dst : 
            dst.write(qc_img_2,1)


This cell simply preprocessed the QC files that you downloaded along with the LST. The QC files are coded in bits and not in integers that could be easily understood by pyDMS as a mask file. That cell transformed these QC files into a Quality Flag files of integer numbers. 
0 means that the pixel was perfect, 1 means nominal quality pixel 2 means cloud detected and 3 means pixel not produced, we'll reject these later.  
For more information on the QC files : https://lpdaac.usgs.gov/documents/423/ECO2_User_Guide_V1.pdf (section 2.4)

Scaling the ECOSTRESS LST to normal Kelvin scale.  
The LST product is actually scaled at 0.02, the GIS takes that scale in account before display so you might not see it. However, Python doesn't so we need to scaled our LST to regular Kelvin.  
Skip this cell if your ECOSTRESS LST data is already scaled.

In [ ]:
for file in os.listdir(lr_dir) :
        if file.endswith('001.tif') : # avoid the auxiliary files created by QGIS 
                with rasterio.open(os.path.join(lr_dir,file),'r') as lr_img: 
                        out_image=lr_img.read()
                        out_meta = lr_img.meta
                
                out_meta.update({"driver": "GTiff",
                        "height": out_image.shape[1],
                        "width": out_image.shape[2],
                        'dtype' :'float32'})    

                if not os.path.exists(lr_dir_sc) :
                        os.mkdir(lr_dir_sc)
                dst_path = os.path.join(lr_dir_sc,file)
                with rasterio.open(dst_path,'w',**out_meta) as dst : 
                        dst.write(out_image*0.002 +0.49)

### Upsampling using pyDMS

The preprocessing is now over. Let's sharpen using pyDMS. Use one of the following cells depending on the presence of Quality Control files and the desired extent.

If you have QC files, process them into QF files and use one these two cells :

If you use this cell, then the output extent will be the extent of the HR image. Thus, if the HLS has a bigger extent, then the edges will be padded with NaNs. If it is smaller, then the sharpened image's extent will be the HLS extent (pyDMS is coded that way)

In [ ]:
useDecisionTree = True # Change this to False if you want to use the Neural Network intead of the Decision tree

files_sharpened = [] #list of the files sharpened

# Loop through the directory of LST images
for file in os.listdir(lr_dir_sc) :
    if file.endswith('sh.tif') :
        if not os.path.exists(dst_dir) :
                        os.mkdir(dst_dir)
        outputFilename = os.path.join(dst_dir,file.replace('.tif','_sharp_HLS.tif')) # destination path for the sharpened images
        
        highResFilename = hr_img_path 
        lowResFilename = os.path.join(lr_dir_sc,file) 
        
        # Locate the corresponding mask file (our QF processed earlier)
        file_qc = 'QC'.join(file.rsplit('LST', 1))
        file_qf = file_qc.replace('.tif','_QF.tif')
        lowResMaskFilename = os.path.join(QC_dir,file_qf)
    
    
        # Only sharpen the files where the proportion of ideal pixels is greater than tresh, here 75%
        with rasterio.open(lowResMaskFilename,'r') as mask : 
            mask_array = mask.read(1)
            mask_sz = mask_array.size
            tresh= 0.75 # adjustable
            
        if np.count_nonzero(mask_array==0) + np.count_nonzero(mask_array==1)>tresh*mask_sz :
            
            commonOpts = {"highResFiles":               [highResFilename],
                            "lowResFiles":                [lowResFilename],
                            "lowResQualityFiles":         [lowResMaskFilename],
                            "lowResGoodQualityFlags":     [0,1],
                            "cvHomogeneityThreshold":     0,
                            "movingWindowSize":           0,
                            "disaggregatingTemperature":  True}
            dtOpts =     {"perLeafLinearRegression":    True,
                            "linearRegressionExtrapolationRatio": 0.25}
            sknnOpts =   {'hidden_layer_sizes':         (10,),
                            'activation':                 'tanh'}
            nnOpts =     {"regressionType":             REG_sklearn_ann,
                            "regressorOpt":               sknnOpts}
            
            start_time = time.time()

            if useDecisionTree:
                opts = commonOpts.copy()
                opts.update(dtOpts)
                disaggregator = DecisionTreeSharpener(**opts)
            else:
                opts = commonOpts.copy()
                opts.update(nnOpts)
                disaggregator = NeuralNetworkSharpener(**opts)

            print("Training regressor...")
            disaggregator.trainSharpener()
            print("Sharpening...")
            downscaledFile = disaggregator.applySharpener(highResFilename, lowResFilename)
            print("Residual analysis...")
            residualImage, correctedImage = disaggregator.residualAnalysis(downscaledFile, lowResFilename,
                                                                            lowResMaskFilename,
                                                                            doCorrection=True)
            print("Saving output...")
            highResFile = gdal.Open(highResFilename)
            if correctedImage is not None:
                outImage = correctedImage
            else:
                outImage = downscaledFile
            outFile = utils.saveImg(outImage.GetRasterBand(1).ReadAsArray(),
                                    outImage.GetGeoTransform(),
                                    outImage.GetProjection(),
                                    outputFilename)
            residualFile = utils.saveImg(residualImage.GetRasterBand(1).ReadAsArray(),
                                            residualImage.GetGeoTransform(),
                                            residualImage.GetProjection(),
                                            os.path.splitext(outputFilename)[0] + "_residual" +
                                            os.path.splitext(outputFilename)[1])
            files_sharpened.append(file)
            outFile = None
            residualFile = None
            downsaceldFile = None
            highResFile = None

            print(time.time() - start_time, "seconds")
        
    
    

If the ECOSTRESS image is completely included in the HLS image, then if desired, it is possible to cut the HLS image to the extent of the ECOSTRESS image, or to an extent of the user's choosing (which also has to be included). This might be useful for any mathematical postprocessing, such as computing RMSE or residual, because such an image would not contain any NaNs, and images would share same extents. To do so, use this cell :

In [ ]:
useDecisionTree = True # Change this to False if you want to use the Neural Network intead of the Decision tree

files_sharpened = [] #list of the files sharpened

# Loop through the directory of LST images
for file in os.listdir(lr_dir_sc) :
    if file.endswith('.tif') :
        if not os.path.exists(dst_dir) :
                        os.mkdir(dst_dir)
        outputFilename = os.path.join(dst_dir,file.replace('.tif','_sharp_HLS.tif')) # destination path for the sharpened images
        lowResFilename = os.path.join(lr_dir_sc,file) 
        lr_ds = rasterio.open(lowResFilename)
        
        projwin = [lr_ds.bounds.left,lr_ds.bounds.top,lr_ds.bounds.right,lr_ds.bounds.bottom] # cut to the extent of the LR image
        # projwin = [-118.08175, 34.16189, -117.998676, 34.112327] # example of a custom cutout over LA 
        

        hr_img_path_clipped = hr_img_path.replace('.tif',f'_clipped.tif') # also produces a clipped version of the HR image
        ds = gdal.Open(hr_img_path)
        ds = gdal.Translate(hr_img_path_clipped, ds, projWin = projwin)
        ds = None
        lr_ds = None
        
        highResFilename = hr_img_path_clipped 
        
        
        # Locate the corresponding mask file (our QF processed earlier)
        file_qc = 'QC'.join(file.rsplit('LST', 1))
        file_qf = file_qc.replace('.tif','_QF.tif')
        lowResMaskFilename = os.path.join(QC_dir,file_qf)
    
    
        # Only sharpen the files where the proportion of ideal pixels is greater than tresh, here 75%
        with rasterio.open(lowResMaskFilename,'r') as mask : 
            mask_array = mask.read(1)
            mask_sz = mask_array.size
            tresh= 0.75 # adjustable
            
        if np.count_nonzero(mask_array==0) + np.count_nonzero(mask_array==1)>tresh*mask_sz :
            
            commonOpts = {"highResFiles":               [highResFilename],
                            "lowResFiles":                [lowResFilename],
                            "lowResQualityFiles":         [lowResMaskFilename],
                            "lowResGoodQualityFlags":     [0,1],
                            "cvHomogeneityThreshold":     0,
                            "movingWindowSize":           0,
                            "disaggregatingTemperature":  True}
            dtOpts =     {"perLeafLinearRegression":    True,
                            "linearRegressionExtrapolationRatio": 0.25}
            sknnOpts =   {'hidden_layer_sizes':         (10,),
                            'activation':                 'tanh'}
            nnOpts =     {"regressionType":             REG_sklearn_ann,
                            "regressorOpt":               sknnOpts}
            
            start_time = time.time()

            if useDecisionTree:
                opts = commonOpts.copy()
                opts.update(dtOpts)
                disaggregator = DecisionTreeSharpener(**opts)
            else:
                opts = commonOpts.copy()
                opts.update(nnOpts)
                disaggregator = NeuralNetworkSharpener(**opts)

            print("Training regressor...")
            disaggregator.trainSharpener()
            print("Sharpening...")
            downscaledFile = disaggregator.applySharpener(highResFilename, lowResFilename)
            print("Residual analysis...")
            residualImage, correctedImage = disaggregator.residualAnalysis(downscaledFile, lowResFilename,
                                                                            lowResMaskFilename,
                                                                            doCorrection=True)
            print("Saving output...")
            highResFile = gdal.Open(highResFilename)
            if correctedImage is not None:
                outImage = correctedImage
            else:
                outImage = downscaledFile
            outFile = utils.saveImg(outImage.GetRasterBand(1).ReadAsArray(),
                                    outImage.GetGeoTransform(),
                                    outImage.GetProjection(),
                                    outputFilename)
            residualFile = utils.saveImg(residualImage.GetRasterBand(1).ReadAsArray(),
                                            residualImage.GetGeoTransform(),
                                            residualImage.GetProjection(),
                                            os.path.splitext(outputFilename)[0] + "_residual" +
                                            os.path.splitext(outputFilename)[1])
            files_sharpened.append(file)
            outFile = None
            residualFile = None
            downsaceldFile = None
            highResFile = None

            print(time.time() - start_time, "seconds")

If you sharpen an image with no QC file, which can happen for instance if this a multi day aggregate, then use of these cells : 

Final product to the HR extent :

In [ ]:
useDecisionTree = True # Change this to False if you want to use the Neural Network intead of the Decision tree

files_sharpened = [] #list of the files sharpened

# Loop through the directory of LST images
for file in os.listdir(lr_dir_sc) :
    if file.endswith('.tif') :
        if not os.path.exists(dst_dir) :
                        os.mkdir(dst_dir)
        outputFilename = os.path.join(dst_dir,file.replace('.tif','_sharp_HLS.tif')) # destination path for the sharpened images
        lowResFilename = os.path.join(lr_dir_sc,file)      
        highResFilename = hr_img_path
        lowResMaskFilename = ''

            
        commonOpts = {"highResFiles":               [highResFilename],
                        "lowResFiles":                [lowResFilename],
                        "lowResQualityFiles":         [lowResMaskFilename],
                        "lowResGoodQualityFlags":     [],
                        "cvHomogeneityThreshold":     0,
                        "movingWindowSize":           0,
                        "disaggregatingTemperature":  False}
        dtOpts =     {"perLeafLinearRegression":    True,
                        "linearRegressionExtrapolationRatio": 0.25}
        sknnOpts =   {'hidden_layer_sizes':         (10,),
                        'activation':                 'tanh'}
        nnOpts =     {"regressionType":             REG_sklearn_ann,
                        "regressorOpt":               sknnOpts}
        
        start_time = time.time()

        if useDecisionTree:
            opts = commonOpts.copy()
            opts.update(dtOpts)
            disaggregator = DecisionTreeSharpener(**opts)
        else:
            opts = commonOpts.copy()
            opts.update(nnOpts)
            disaggregator = NeuralNetworkSharpener(**opts)

        print("Training regressor...")
        disaggregator.trainSharpener()
        print("Sharpening...")
        downscaledFile = disaggregator.applySharpener(highResFilename, lowResFilename)
        print("Residual analysis...")
        residualImage, correctedImage = disaggregator.residualAnalysis(downscaledFile, lowResFilename,
                                                                        lowResMaskFilename,
                                                                        doCorrection=True)
        print("Saving output...")
        highResFile = gdal.Open(highResFilename)
        if correctedImage is not None:
            outImage = correctedImage
        else:
            outImage = downscaledFile
        outFile = utils.saveImg(outImage.GetRasterBand(1).ReadAsArray(),
                                outImage.GetGeoTransform(),
                                outImage.GetProjection(),
                                outputFilename)
        residualFile = utils.saveImg(residualImage.GetRasterBand(1).ReadAsArray(),
                                        residualImage.GetGeoTransform(),
                                        residualImage.GetProjection(),
                                        os.path.splitext(outputFilename)[0] + "_residual" +
                                        os.path.splitext(outputFilename)[1])
        files_sharpened.append(file)
        outFile = None
        residualFile = None
        downsaceldFile = None
        highResFile = None

        print(time.time() - start_time, "seconds")

Final product to a smaller extent :

In [ ]:
useDecisionTree = True # Change this to False if you want to use the Neural Network intead of the Decision tree

files_sharpened = [] #list of the files sharpened

# Loop through the directory of LST images
for file in os.listdir(lr_dir_sc) :
    if file.endswith('.tif') :
        if not os.path.exists(dst_dir) :
                        os.mkdir(dst_dir)
        outputFilename = os.path.join(dst_dir,file.replace('.tif','_sharp_HLS.tif')) # destination path for the sharpened images
        lowResFilename = os.path.join(lr_dir_sc,file)         
        lr_ds = rasterio.open(lowResFilename)
        
        projwin = [lr_ds.bounds.left,lr_ds.bounds.top,lr_ds.bounds.right,lr_ds.bounds.bottom] # cut to the extent of the LR image
        # projwin = [-118.08175, 34.16189, -117.998676, 34.112327] # example of a custom cutout over LA 
        

        hr_img_path_clipped = hr_img_path.replace('.tif',f'_clipped.tif') # also produces a clipped version of the HR image
        ds = gdal.Open(hr_img_path)
        ds = gdal.Translate(hr_img_path_clipped, ds, projWin = projwin)
        ds = None
        lr_ds = None
        
        highResFilename = hr_img_path_clipped
        lowResMaskFilename = ''

            
        commonOpts = {"highResFiles":               [highResFilename],
                        "lowResFiles":                [lowResFilename],
                        "lowResQualityFiles":         [lowResMaskFilename],
                        "lowResGoodQualityFlags":     [],
                        "cvHomogeneityThreshold":     0,
                        "movingWindowSize":           0,
                        "disaggregatingTemperature":  False}
        dtOpts =     {"perLeafLinearRegression":    True,
                        "linearRegressionExtrapolationRatio": 0.25}
        sknnOpts =   {'hidden_layer_sizes':         (10,),
                        'activation':                 'tanh'}
        nnOpts =     {"regressionType":             REG_sklearn_ann,
                        "regressorOpt":               sknnOpts}
        
        start_time = time.time()

        if useDecisionTree:
            opts = commonOpts.copy()
            opts.update(dtOpts)
            disaggregator = DecisionTreeSharpener(**opts)
        else:
            opts = commonOpts.copy()
            opts.update(nnOpts)
            disaggregator = NeuralNetworkSharpener(**opts)

        print("Training regressor...")
        disaggregator.trainSharpener()
        print("Sharpening...")
        downscaledFile = disaggregator.applySharpener(highResFilename, lowResFilename)
        print("Residual analysis...")
        residualImage, correctedImage = disaggregator.residualAnalysis(downscaledFile, lowResFilename,
                                                                        lowResMaskFilename,
                                                                        doCorrection=True)
        print("Saving output...")
        highResFile = gdal.Open(highResFilename)
        if correctedImage is not None:
            outImage = correctedImage
        else:
            outImage = downscaledFile
        
        outFile = utils.saveImg(outImage.GetRasterBand(1).ReadAsArray(),
                                outImage.GetGeoTransform(),
                                outImage.GetProjection(),
                                outputFilename)
        residualFile = utils.saveImg(residualImage.GetRasterBand(1).ReadAsArray(),
                                        residualImage.GetGeoTransform(),
                                        residualImage.GetProjection(),
                                        os.path.splitext(outputFilename)[0] + "_residual" +
                                        os.path.splitext(outputFilename)[1])
        files_sharpened.append(file)
        outFile = None
        residualFile = None
        downsaceldFile = None
        highResFile = None

        print(time.time() - start_time, "seconds")

### Display

Plot one random sharpened image

In [ ]:
# Pick one random sharpened file
file = random.choice(files_sharpened)
sharpened_file = os.path.join(dst_dir,file.replace('.tif','_sharp_HLS.tif'))
raw_file = os.path.join(lr_dir_sc,file)
with rasterio.open(raw_file,'r') as lr_img : 
    raw_lst = lr_img.read(1)
with rasterio.open(sharpened_file,'r') as shrp_img :
    shrp_lst = shrp_img.read(1)
    
# Plot the ECOSTRESS original product
plt.figure(1)
plt.imshow(raw_lst,cmap='viridis')
plt.axis('off')
plt.colorbar(label ='LST(K)')
vmin, vmax = plt.gci().get_clim() # save the limits to share them in the second figure
plt.title("ECOSTRESS LST 70m")
plt.show()

# Plot the ECOSTRESS sharpened product
plt.figure(2)
plt.imshow(shrp_lst,cmap='viridis',clim =(vmin,vmax))
plt.axis('off')
plt.colorbar(label ='LST(K)')
plt.title("ECOSTRESS LST sharpened to 30m")
plt.show()